In [93]:
import os
import io
import uuid
import time
import shutil
import requests
import pandas as pd

from Bio import SeqIO
from Bio import pairwise2
from Bio.PDB.MMCIFParser import MMCIFParser

In [ ]:
df = pd.read_csv("pharbp_export.csv")
df.head(5)

C:\Users\danie\AppData\Local\Temp\ipykernel_18484\1050770431.py:3: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("pharbp_export.csv")


,Protein ID,Protein Name,Type (Phage / Prophage),Evidence Class(es),Evidence Type(s),Evidence Probability(ies),Source,Source ID,Phage Name,Host Name(s),CDS ID
0,1,Tail spike protein,phage,TSP,UniProt/Swiss,1,UniParc,UPI0000DAD899,Bacillus phage SPP1,Bacillus subtilis,CAL18688.1
1,2,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9491, 0.9523, 1.0000",UniParc,UPI00022AB2F7,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94255.1
2,3,Phage tailspike protein,phage,TSP,"DepoScope, GenBank, Phold, RBPdetect, RBPdetec...","0.9478, 0.9556, 1.0000",UniParc,UPI00022AB2F9,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94257.1
3,4,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9377, 0.9417, 1.0000",UniParc,UPI00022AB2FA,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94258.1
4,5,Tail spike protein 1,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9457, 0.9531, 0.9997, 1.0000",UniParc,UPI00023306CF,Salmonella typhimurium phage PhiSH19,Salmonella enterica subsp. enterica serovar Ty...,AER70268.1


In [2]:
df["Evidence Count"] = df["Evidence Type(s)"].apply(
    lambda x: len([t.strip() for t in str(x).split(",")])
)
df.head(5)

,Protein ID,Protein Name,Type (Phage / Prophage),Evidence Class(es),Evidence Type(s),Evidence Probability(ies),Source,Source ID,Phage Name,Host Name(s),CDS ID,Evidence Count
0,1,Tail spike protein,phage,TSP,UniProt/Swiss,1,UniParc,UPI0000DAD899,Bacillus phage SPP1,Bacillus subtilis,CAL18688.1,1
1,2,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9491, 0.9523, 1.0000",UniParc,UPI00022AB2F7,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94255.1,4
2,3,Phage tailspike protein,phage,TSP,"DepoScope, GenBank, Phold, RBPdetect, RBPdetec...","0.9478, 0.9556, 1.0000",UniParc,UPI00022AB2F9,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94257.1,6
3,4,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9377, 0.9417, 1.0000",UniParc,UPI00022AB2FA,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94258.1,4
4,5,Tail spike protein 1,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9457, 0.9531, 0.9997, 1.0000",UniParc,UPI00023306CF,Salmonella typhimurium phage PhiSH19,Salmonella enterica subsp. enterica serovar Ty...,AER70268.1,4


In [3]:
def parse_probs(x):
    return [float(v.strip()) for v in str(x).split(",")]
    
df["Evidence Prob List"] = df["Evidence Probability(ies)"].apply(parse_probs)
df.head(5)

,Protein ID,Protein Name,Type (Phage / Prophage),Evidence Class(es),Evidence Type(s),Evidence Probability(ies),Source,Source ID,Phage Name,Host Name(s),CDS ID,Evidence Count,Evidence Prob List
0,1,Tail spike protein,phage,TSP,UniProt/Swiss,1,UniParc,UPI0000DAD899,Bacillus phage SPP1,Bacillus subtilis,CAL18688.1,1,[1.0]
1,2,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9491, 0.9523, 1.0000",UniParc,UPI00022AB2F7,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94255.1,4,"[0.9491, 0.9523, 1.0]"
2,3,Phage tailspike protein,phage,TSP,"DepoScope, GenBank, Phold, RBPdetect, RBPdetec...","0.9478, 0.9556, 1.0000",UniParc,UPI00022AB2F9,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94257.1,6,"[0.9478, 0.9556, 1.0]"
3,4,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9377, 0.9417, 1.0000",UniParc,UPI00022AB2FA,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94258.1,4,"[0.9377, 0.9417, 1.0]"
4,5,Tail spike protein 1,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9457, 0.9531, 0.9997, 1.0000",UniParc,UPI00023306CF,Salmonella typhimurium phage PhiSH19,Salmonella enterica subsp. enterica serovar Ty...,AER70268.1,4,"[0.9457, 0.9531, 0.9997, 1.0]"


In [4]:
df["All Probs ≥ 0.95"] = df["Evidence Prob List"].apply(
    lambda lst: all(v >= 0.95 for v in lst)
)
df.head(5)

,Protein ID,Protein Name,Type (Phage / Prophage),Evidence Class(es),Evidence Type(s),Evidence Probability(ies),Source,Source ID,Phage Name,Host Name(s),CDS ID,Evidence Count,Evidence Prob List,All Probs ≥ 0.95
0,1,Tail spike protein,phage,TSP,UniProt/Swiss,1,UniParc,UPI0000DAD899,Bacillus phage SPP1,Bacillus subtilis,CAL18688.1,1,[1.0],True
1,2,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9491, 0.9523, 1.0000",UniParc,UPI00022AB2F7,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94255.1,4,"[0.9491, 0.9523, 1.0]",False
2,3,Phage tailspike protein,phage,TSP,"DepoScope, GenBank, Phold, RBPdetect, RBPdetec...","0.9478, 0.9556, 1.0000",UniParc,UPI00022AB2F9,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94257.1,6,"[0.9478, 0.9556, 1.0]",False
3,4,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9377, 0.9417, 1.0000",UniParc,UPI00022AB2FA,Salmonella phage SFP10,"Escherichia coli O157, Salmonella sp.",AEN94258.1,4,"[0.9377, 0.9417, 1.0]",False
4,5,Tail spike protein 1,phage,TSP,"DepoScope, RBPdetect, RBPdetect2, UniProt/TrEMBL","0.9457, 0.9531, 0.9997, 1.0000",UniParc,UPI00023306CF,Salmonella typhimurium phage PhiSH19,Salmonella enterica subsp. enterica serovar Ty...,AER70268.1,4,"[0.9457, 0.9531, 0.9997, 1.0]",False


In [5]:
df_filtered = df[
    (df["Evidence Count"] >= 3) &
    (df["All Probs ≥ 0.95"])
]
df_filtered.head(5)

,Protein ID,Protein Name,Type (Phage / Prophage),Evidence Class(es),Evidence Type(s),Evidence Probability(ies),Source,Source ID,Phage Name,Host Name(s),CDS ID,Evidence Count,Evidence Prob List,All Probs ≥ 0.95
29,30,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, Phold, RBPdetect, RBPdetect2, UniPr...","0.9500, 0.9516, 1.0000",UniParc,UPI00028A4B5F,Salmonella phage SKML-39,Salmonella,AFU64345.1,5,"[0.95, 0.9516, 1.0]",True
48,49,Endosialidase tailspike,phage,TSP,"GenBank, Phold, UniProt/TrEMBL",1,UniParc,UPI000182A229,Bacillus phage CampHawk,Bacillus subtilis,AGY46939.1,3,[1.0],True
504,505,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect2, UniProt/TrEMBL","0.9502, 1.0000",UniParc,UPI00185F4041,Escherichia phage Mt1B1_P17,NaN,QNJ49293.1,3,"[0.9502, 1.0]",True
605,606,Tail spike protein,phage,TSP,"GenBank, Phold, UniProt/TrEMBL",1,UniParc,UPI001AF91486,Salmonella phage vB_SalP_TR2,NaN,QSJ04043.1,3,[1.0],True
627,628,Tailspike protein,phage,TSP,"GenBank, Phold, UniProt/TrEMBL",1,UniParc,UPI001C756C14,Escherichia phage U1G,NaN,QXV71913.1,3,[1.0],True


In [6]:
print(df_filtered.shape)
print(df_filtered.value_counts("Evidence Count"))

(443, 14)
Evidence Count
3    247
4    195
5      1
Name: count, dtype: int64


In [7]:
df_filtered = df_filtered.dropna(subset=["CDS ID"])
df_filtered.head(5)

,Protein ID,Protein Name,Type (Phage / Prophage),Evidence Class(es),Evidence Type(s),Evidence Probability(ies),Source,Source ID,Phage Name,Host Name(s),CDS ID,Evidence Count,Evidence Prob List,All Probs ≥ 0.95
29,30,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, Phold, RBPdetect, RBPdetect2, UniPr...","0.9500, 0.9516, 1.0000",UniParc,UPI00028A4B5F,Salmonella phage SKML-39,Salmonella,AFU64345.1,5,"[0.95, 0.9516, 1.0]",True
48,49,Endosialidase tailspike,phage,TSP,"GenBank, Phold, UniProt/TrEMBL",1,UniParc,UPI000182A229,Bacillus phage CampHawk,Bacillus subtilis,AGY46939.1,3,[1.0],True
504,505,Tail spike TSP1/Gp66 N-terminal domain-contain...,phage,TSP,"DepoScope, RBPdetect2, UniProt/TrEMBL","0.9502, 1.0000",UniParc,UPI00185F4041,Escherichia phage Mt1B1_P17,NaN,QNJ49293.1,3,"[0.9502, 1.0]",True
605,606,Tail spike protein,phage,TSP,"GenBank, Phold, UniProt/TrEMBL",1,UniParc,UPI001AF91486,Salmonella phage vB_SalP_TR2,NaN,QSJ04043.1,3,[1.0],True
627,628,Tailspike protein,phage,TSP,"GenBank, Phold, UniProt/TrEMBL",1,UniParc,UPI001C756C14,Escherichia phage U1G,NaN,QXV71913.1,3,[1.0],True


In [8]:
print(df_filtered.shape)
print(df_filtered.value_counts("Evidence Count"))

(432, 14)
Evidence Count
3    236
4    195
5      1
Name: count, dtype: int64


In [ ]:
cds_ids = (
    df_filtered["CDS ID"]
    .dropna()
    .apply(lambda x: [i.strip() for i in str(x).split(",")])
    .explode()
    .unique()
    .tolist()
)

print("Number of unique CDS IDs:", len(cds_ids))
print(cds_ids[:10]) 

Number of unique CDS IDs: 437
['AFU64345.1', 'AGY46939.1', 'QNJ49293.1', 'QSJ04043.1', 'QXV71913.1', 'AAD25465.1', 'AGM10738.1', 'AEX65881.1', 'AEW24664.1', 'AFA44869.1']


In [ ]:
fasta_path = "pharbp_sequences.fasta"   

seq_records = SeqIO.to_dict(SeqIO.parse(fasta_path, "fasta"))

In [11]:
seq_records.keys()

dict_keys(['QDJ98910.1', 'YP_010298710.1', 'WWS23899.1', 'QHB48481.1', 'QBI82700.1', 'WNT46376.1', 'CAM0024795.1', 'AIX42482.1', 'AAX91899.1', 'WGH28304.1', 'AXQ63991.1', 'AIX44951.1', 'CAM0047267.1', 'UIS74187.1', 'CAD44229.1', 'CAB5079010.1', 'UCR74084.1', 'AXY85422.1', 'WIL00752.1', 'UXQ84694.1', 'WDS61933.1', 'USW07479.1', 'AYJ75173.1', 'QDH91899.1', 'QGT54279.1', 'QZI83589.1', 'ANZ48815.1', 'QUL76918.1', 'AHG24049.1', 'WNA13705.1', 'UPW37470.1', 'QJB22091.1', 'CAB4169900.1', 'QEG04139.1', 'YP_010299872.1', 'QOV06361.1', 'VCU43532.1', 'QAY06789.1', 'YP_009849665.1', 'AIW03318.1', 'WPK35244.1', 'CAB4160854.1', 'XAF37120.1', 'AKO59828.1', 'URC09674.1', 'WNM52995.1', 'AOO07447.1', 'XDN94342.1', 'QBP05921.1', 'XFC50159.1', 'AFK66644.1', 'WPJ54721.1', 'AEQ39207.1', 'APC45943.1', 'QGH74310.1', 'WAX07893.1', 'QBX28292.1', 'YP_010746423.1', 'XRH82125.1', 'YP_010106982.1', 'QXN70848.1', 'QDF19416.1', 'WNV46782.1', 'AUR81018.1', 'CAM0104536.1', 'QZE56985.1', 'AGY37073.1', 'AOT27424.1', 'ALA4

In [ ]:
def normalize_cds_id(cds):
    
    cds = cds.strip()
    protected_prefixes = ("YP_", "XP_", "WP_", "NP_", "AP_")

    if cds.startswith(protected_prefixes):
        return cds  
    
    return cds.split("_")[0]

In [ ]:
cds_to_seq = {}

for cds in cds_ids:
    norm = normalize_cds_id(cds)
    if norm in seq_records:
        cds_to_seq[cds] = str(seq_records[norm].seq)
    else:
        print("WARNING: normalized CDS ID not found in FASTA:", cds, "->", norm)

print("Sequences found:", len(cds_to_seq))

Sequences found: 437


In [14]:
cds_to_seq["AAD25465.1"] #Exanple

'MPHGFRVFFSICRSGLVTGRGRMSRGWMKWAVIQAEQENDMNILRKLMQSLCGCGKHDDCENGQSLTAQLRLGPADILESDENGIIPEQDRVITQVVILDADKKQIQCVVRPLQILRADGTWENIGGMK'

In [ ]:
def rcsb_search_sequence(sequence, retries=3):
    
    url = "https://search.rcsb.org/rcsbsearch/v2/query?json"

    payload = {
        "query": {
            "type": "terminal",
            "service": "sequence",
            "parameters": {
                "value": sequence,
                "sequence_type": "protein",
                "evalue_cutoff": 10,
                "identity_cutoff": 0.0,
                "target": "pdb_protein_sequence"
            }
        },
        "return_type": "polymer_entity",
        "request_info": {
            "query_id": str(uuid.uuid4())
        },
        "request_options": {
            "paginate": {
                "start": 0,
                "rows": 100
            }
        }
    }

    for attempt in range(retries):
        r = requests.post(url, json=payload)

        if r.status_code == 200:
            data = r.json()
            hits = []
            for hit in data.get("result_set", []):
                identifier = hit["identifier"]
                score = hit.get("score")
                identity = score.get("sequence_identity") if isinstance(score, dict) else None
                pdb_id, entity_id = identifier.split("_")
                hits.append({
                    "pdb_id": pdb_id,
                    "entity_id": entity_id,
                    "identity": identity
                })
            return hits
        
 
        print("\nRCSB ERROR")
        print("Status:", r.status_code)
        print("Reason:", r.reason)
        print("Response:", repr(r.text))

        time.sleep(1)  

    return []  

In [ ]:
all_hits = {}

for cds_id, seq in cds_to_seq.items():
    print("Searching:", cds_id)
    hits = rcsb_search_sequence(seq)
    all_hits[cds_id] = hits
    time.sleep(0.2) 

Searching: AFU64345.1
Searching: AGY46939.1

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''
Searching: QNJ49293.1
Searching: QSJ04043.1
Searching: QXV71913.1
Searching: AAD25465.1

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''
Searching: AGM10738.1
Searching: AEX65881.1

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''
Searching: AEW24664.1

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ERROR
Status: 204
Reason: No Content
Response: ''
Searching: AFA44869.1
Searching: AFD10622.1
Searching: AFJ75971.1

RCSB ERROR
Status: 204
Reason: No Content
Response: ''

RCSB ER

In [18]:
for cds, hits in all_hits.items():
    print(cds, hits)

AFU64345.1 [{'pdb_id': '4OJ5', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJ6', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJL', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJO', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJP', 'entity_id': '1', 'identity': None}, {'pdb_id': '9D84', 'entity_id': '1', 'identity': None}, {'pdb_id': '4QNL', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6F', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6H', 'entity_id': '1', 'identity': None}, {'pdb_id': '6NW9', 'entity_id': '1', 'identity': None}, {'pdb_id': '9D80', 'entity_id': '4', 'identity': None}, {'pdb_id': '9D81', 'entity_id': '1', 'identity': None}, {'pdb_id': '9OCC', 'entity_id': '5', 'identity': None}, {'pdb_id': '6W4Q', 'entity_id': '1', 'identity': None}, {'pdb_id': '8X8M', 'entity_id': '1', 'identity': None}, {'pdb_id': '8X8O', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6P', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6S', 'entity_id': '1',

In [19]:
all_hits_nonempty = {cds: hits for cds, hits in all_hits.items() if len(hits) > 0}
print("Original number of entries:", len(all_hits))
print("Number of entries with at least one hit:", len(all_hits_nonempty))
empty_entries = [cds for cds, hits in all_hits.items() if len(hits) == 0]
print("Entries removed (no hits):", len(empty_entries))

Original number of entries: 437
Number of entries with at least one hit: 339
Entries removed (no hits): 98


In [20]:
for cds, hits in all_hits_nonempty.items():
    print(cds, hits)

AFU64345.1 [{'pdb_id': '4OJ5', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJ6', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJL', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJO', 'entity_id': '1', 'identity': None}, {'pdb_id': '4OJP', 'entity_id': '1', 'identity': None}, {'pdb_id': '9D84', 'entity_id': '1', 'identity': None}, {'pdb_id': '4QNL', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6F', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6H', 'entity_id': '1', 'identity': None}, {'pdb_id': '6NW9', 'entity_id': '1', 'identity': None}, {'pdb_id': '9D80', 'entity_id': '4', 'identity': None}, {'pdb_id': '9D81', 'entity_id': '1', 'identity': None}, {'pdb_id': '9OCC', 'entity_id': '5', 'identity': None}, {'pdb_id': '6W4Q', 'entity_id': '1', 'identity': None}, {'pdb_id': '8X8M', 'entity_id': '1', 'identity': None}, {'pdb_id': '8X8O', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6P', 'entity_id': '1', 'identity': None}, {'pdb_id': '5W6S', 'entity_id': '1',

In [ ]:
def get_pdb_sequence(pdb_id, entity_id):
    url = f"https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/{entity_id}"
    r = requests.get(url)

    if r.status_code != 200:
        print(f"WARNING: Failed to fetch PDB {pdb_id} entity {entity_id} (HTTP {r.status_code})")
        return None

    data = r.json()


    if "entity_poly" not in data or "pdbx_seq_one_letter_code_can" not in data["entity_poly"]:
        print(f"WARNING: Sequence missing in PDB {pdb_id} entity {entity_id}")
        return None

    seq = data["entity_poly"]["pdbx_seq_one_letter_code_can"]
    seq = "".join(seq.split())  


    if len(seq) == 0:
        print(f"WARNING: Empty sequence for PDB {pdb_id} entity {entity_id}")
        return None

    return seq

In [ ]:
def compute_identity_and_coverage(query, target):

    aln = pairwise2.align.localms(query, target, 2, -1, -5, -1, one_alignment_only=True)[0]

    aligned_q = aln.seqA
    aligned_t = aln.seqB

    matches = sum(1 for a, b in zip(aligned_q, aligned_t) if a == b)
    identity = matches / len(aligned_q)

    aligned_region_length = aln.end - aln.start
    coverage = aligned_region_length / len(query)

    if coverage > 1:
        coverage = 1.0

    return identity, coverage

c:\Users\danie\Desktop\pre-interview task\venv\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [ ]:
def evaluate_hits_for_cds(cds_id, cds_seq, hits):
    results = []

    for hit in hits:
        pdb_id = hit["pdb_id"]
        entity_id = hit["entity_id"]

        pdb_seq = get_pdb_sequence(pdb_id, entity_id)
        if pdb_seq is None:
            continue

        identity, coverage = compute_identity_and_coverage(cds_seq, pdb_seq)

        results.append({
            "cds_id": cds_id,
            "pdb_id": pdb_id,
            "entity_id": entity_id,
            "identity": identity,
            "coverage": coverage
        })

    return results

In [ ]:
def filter_pdb_hits(results, identity_threshold=0.50, coverage_threshold=0.70):
    """Return ALL PDB hits with printed identity + coverage values."""
    
    for r in results:
        ident = r["identity"]
        cov = r["coverage"]
        print(f"CHECK: {r['pdb_id']}_{r['entity_id']} "
              f"identity={ident}, coverage={cov}")

    filtered = [
        r for r in results
        if r["identity"] is not None
        and r["coverage"] is not None
        and r["identity"] >= identity_threshold
        and r["coverage"] >= coverage_threshold
    ]

    return filtered

In [26]:
total = len(all_hits_nonempty)
count = 0
final_templates = {}


for i, (cds_id, hits) in enumerate(all_hits_nonempty.items(), start=1):


    cds_seq = cds_to_seq[cds_id]
    evaluated = evaluate_hits_for_cds(cds_id, cds_seq, hits)
    passing = filter_pdb_hits(evaluated, identity_threshold=0.4, coverage_threshold=0.7)


    final_templates[cds_id] = {
        "passing_hits": passing,
        "all_hits": evaluated
    }


    print(f"[{i}/{total}] Processed {cds_id} — passing hits: {len(passing)}")


num_with_templates = sum(
    1 for v in final_templates.values() if len(v["passing_hits"]) > 0
)

print("\nEntries with ≥1 good PDB template:", num_with_templates)
print("Total entries processed:", total)

CHECK: 4OJ5_1 identity=0.21778350515463918, coverage=0.23595505617977527
CHECK: 4OJ6_1 identity=0.21778350515463918, coverage=0.23595505617977527
CHECK: 4OJL_1 identity=0.21778350515463918, coverage=0.23595505617977527
CHECK: 4OJO_1 identity=0.21778350515463918, coverage=0.23595505617977527
CHECK: 4OJP_1 identity=0.21778350515463918, coverage=0.23595505617977527
CHECK: 9D84_1 identity=0.19176319176319176, coverage=0.22612359550561797
CHECK: 4QNL_1 identity=0.1769499417927823, coverage=0.21207865168539325
CHECK: 5W6F_1 identity=0.18398876404494383, coverage=0.21067415730337077
CHECK: 5W6H_1 identity=0.18258426966292135, coverage=0.19803370786516855
CHECK: 6NW9_1 identity=0.18398876404494383, coverage=0.21067415730337077
CHECK: 9D80_4 identity=0.058011049723756904, coverage=0.056179775280898875
CHECK: 9D81_1 identity=0.058011049723756904, coverage=0.056179775280898875
CHECK: 9OCC_5 identity=0.08567415730337079, coverage=0.056179775280898875
CHECK: 6W4Q_1 identity=0.08306364617044229, cov

In [ ]:
BACTERIA_RANKS = ["bacteria"]
VIRUSES_RANKS = ["viruses"]

def classify_pdb_from_cif(pdb_id):
    """
    Classification rules:
        • phage_only      → only viral protein chains detected
        • phage_bacteria  → viral AND bacterial protein chains present
    """

    url = f"https://files.rcsb.org/download/{pdb_id}.cif"
    r = requests.get(url)
    if r.status_code != 200:
        print(f"WARNING: cannot fetch CIF for {pdb_id}")
        return "phage_only"


    parser = MMCIFParser(QUIET=True)
    struct = parser.get_structure(pdb_id, io.StringIO(r.text))
    cif = parser._mmcif_dict

    entity_types = cif.get("_entity.type", [])
    organism_scientific = cif.get("_entity_src_gen.organism_scientific", [])
    taxonomy_ids = cif.get("_entity_src_gen.ncbi_taxonomy_id", [])

    found_phage = False
    found_bacteria = False

    for i, etype in enumerate(entity_types):

        if not etype.lower().startswith("polymer"):
            continue 

        sci_name = (
            organism_scientific[i].lower()
            if i < len(organism_scientific)
            else ""
        )
        tax_id = taxonomy_ids[i] if i < len(taxonomy_ids) else None


        if tax_id not in (None, ".", "?"):
            try:
                t = requests.get(
                    f"https://www.ebi.ac.uk/ena/taxonomy/rest/tax-id/{tax_id}"
                ).json()
                superk = t.get("superkingdom", "").lower()
            except:
                superk = ""
        else:
            superk = ""


        if superk in VIRUSES_RANKS:
            found_phage = True
        elif superk in BACTERIA_RANKS:
            found_bacteria = True

    if found_phage and found_bacteria:
        return "phage_bacteria"

    return "phage_only"

In [ ]:
def download_structure(pdb_id):
    """Download PDB file; if missing, download CIF file instead."""


    url_pdb = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    r = requests.get(url_pdb)

    if r.status_code == 200:
        return r.text, ".pdb"

    url_cif = f"https://files.rcsb.org/download/{pdb_id}.cif"
    r2 = requests.get(url_cif)

    if r2.status_code == 200:
        return r2.text, ".cif"

    print(f"WARNING: Cannot download PDB or CIF for {pdb_id}")
    return None, None


In [87]:
def collect_unique_pdb_ids(final_templates):
    pdb_set = set()

    for cds_id, data in final_templates.items():
        for hit in data.get("passing_hits", []):
            pdb_set.add(hit["pdb_id"])

    return pdb_set


In [ ]:
def download_and_sort_pdbs(pdb_ids, base_dir="results"):
    phage_only_dir = os.path.join(base_dir, "phage_only")
    phage_bac_dir = os.path.join(base_dir, "phage_bacteria")

    os.makedirs(phage_only_dir, exist_ok=True)
    os.makedirs(phage_bac_dir, exist_ok=True)

    pdb_class = {}
    download_cache = {}

    for pdb_id in pdb_ids:
        print(f"\nProcessing PDB {pdb_id}...")


        if pdb_id not in download_cache:
            pdb_text, ext = download_structure(pdb_id)
            if pdb_text is None:
                print(f"  FAILED to download {pdb_id}")
                continue
            download_cache[pdb_id] = (pdb_text, ext)
        else:
            pdb_text, ext = download_cache[pdb_id]


        class_type = classify_pdb_from_cif(pdb_id)
        pdb_class[pdb_id] = class_type

        dest_dir = phage_only_dir if class_type == "phage_only" else phage_bac_dir

        filepath = os.path.join(dest_dir, f"{pdb_id}{ext}")
        with open(filepath, "w") as f:
            f.write(pdb_text)

        print(f"  Saved → {filepath}")
        print(f"  Class → {class_type}")

    return pdb_class

In [ ]:
def build_final_csv(final_templates, pdb_class, output="final_templates.csv"):
    rows = []

    for cds_id, data in final_templates.items():
        for hit in data.get("passing_hits", []):
            pdb_id = hit["pdb_id"]
            identity = hit["identity"]
            coverage = hit["coverage"]

            rows.append({
                "CDS_ID": cds_id,
                "PDB_ID": pdb_id,
                "Identity": identity,
                "Coverage": coverage,
                "Class": pdb_class.get(pdb_id, "unknown")
            })

    df = pd.DataFrame(rows)
    df.to_csv(output, index=False)
    print(f"\nCSV saved → {output}")

    return df

In [ ]:
pdb_ids = collect_unique_pdb_ids(final_templates)
print("Unique PDB count:", len(pdb_ids))

pdb_class = download_and_sort_pdbs(pdb_ids)

df = build_final_csv(final_templates, pdb_class)

Unique PDB count: 37

Processing PDB 7BOX...
  Saved → results\phage_only\7BOX.pdb
  Class → phage_only

Processing PDB 9B45...
  Saved → results\phage_only\9B45.cif
  Class → phage_only

Processing PDB 1V0E...
  Saved → results\phage_only\1V0E.pdb
  Class → phage_only

Processing PDB 7Y1C...
  Saved → results\phage_only\7Y1C.pdb
  Class → phage_only

Processing PDB 8EON...
  Saved → results\phage_only\8EON.cif
  Class → phage_only

Processing PDB 9EUJ...
  Saved → results\phage_only\9EUJ.pdb
  Class → phage_only

Processing PDB 6R21...
  Saved → results\phage_only\6R21.cif
  Class → phage_only

Processing PDB 9B42...
  Saved → results\phage_only\9B42.pdb
  Class → phage_only

Processing PDB 9EUI...
  Saved → results\phage_only\9EUI.cif
  Class → phage_only

Processing PDB 8HQZ...
  Saved → results\phage_only\8HQZ.cif
  Class → phage_only

Processing PDB 7BOY...
  Saved → results\phage_only\7BOY.pdb
  Class → phage_only

Processing PDB 9EUK...
  Saved → results\phage_only\9EUK.pdb
  Cl